# Linear Regression

## Documentation Example

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
training = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
lr = LinearRegression(
    featuresCol='features',
    labelCol='label',
    predictionCol='prediction'
)

In [0]:
lrModel = lr.fit(training)

In [0]:
lrModel.coefficients

Out[8]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lrModel.intercept

Out[9]: 0.14228558260358093

In [0]:
training_summary = lrModel.summary

In [0]:
training_summary.rootMeanSquaredError

Out[11]: 10.16309157133015

# Make a Train-Test Split

In [0]:
all_data = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                360|
|   mean|0.19637111998171833|
| stddev| 10.380046799415004|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_results = correct_model.evaluate(test_data)

# Apply to Unlabeled Data

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = correct_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-1.3070570075053312|
|(10,[0,1,2,3,4,5,...|  5.484667835775298|
|(10,[0,1,2,3,4,5,...|-1.8536221372153079|
|(10,[0,1,2,3,4,5,...| 0.7112255327544701|
|(10,[0,1,2,3,4,5,...| 2.2690798347484855|
|(10,[0,1,2,3,4,5,...| 1.6197003913722638|
|(10,[0,1,2,3,4,5,...| 1.4760467105383352|
|(10,[0,1,2,3,4,5,...|   2.31198651909801|
|(10,[0,1,2,3,4,5,...|  1.478239688815126|
|(10,[0,1,2,3,4,5,...|0.33566939601274376|
|(10,[0,1,2,3,4,5,...| 2.0970519175202114|
|(10,[0,1,2,3,4,5,...| 1.2638851478261297|
|(10,[0,1,2,3,4,5,...|   3.07106797041563|
|(10,[0,1,2,3,4,5,...|  4.940474781277145|
|(10,[0,1,2,3,4,5,...|  1.763853146512657|
|(10,[0,1,2,3,4,5,...| 1.0543283054700978|
|(10,[0,1,2,3,4,5,...|-1.0558759707355658|
|(10,[0,1,2,3,4,5,...| 3.3733998699438876|
|(10,[0,1,2,3,4,5,...| 1.2942773612970637|
|(10,[0,1,2,3,4,5,...| 3.8089165980283743|
+----------